In [1]:
import akshare as ak
import pandas as pd

In [26]:
code = '600746'
stock = ak.stock_zh_a_hist(symbol=code, 
                        period="daily",
                        start_date="20231001",
                        end_date='20250325', 
                        adjust="")

In [27]:
stock['EMA21'] = stock['收盘'].ewm(span=21, min_periods=0, adjust=False, ignore_na=False).mean()
stock['EMA55'] = stock['收盘'].ewm(span=55, min_periods=0, adjust=False, ignore_na=False).mean()
stock = stock[['日期','股票代码','开盘','收盘','最高','最低','成交量','振幅','涨跌幅','涨跌额','EMA21','EMA55']]
stock.tail(3)

,日期,股票代码,开盘,收盘,最高,最低,成交量,振幅,涨跌幅,涨跌额,EMA21,EMA55
352,2025-03-21,600746,7.36,7.35,7.46,7.33,37383,1.76,-0.54,-0.04,7.353323,7.233354
353,2025-03-24,600746,7.31,7.39,7.42,7.28,46524,1.90,0.54,0.04,7.356657,7.238949
354,2025-03-25,600746,7.36,7.69,7.77,7.36,169363,5.55,4.06,0.30,7.386961,7.255058


In [28]:
df = stock.copy()

In [29]:
# 入场买点
def buy(df):
    buy = []
    for i in range(65,len(df)-1):
        if all ([df['EMA21'][i-1] < df['EMA55'][i-1],\
                df['EMA21'][i] > df['EMA55'][i]]):
            buy.append(i)
        else:
            continue
    return buy

In [30]:
def sale(b):
    for w in range(b+1,len(df)):
        # 完全跌破EMA21
        if all([df['开盘'][w]<df['EMA21'][w],\
                df['收盘'][w]<df['EMA21'][w]]):
            s = w
            break
        else:
            s = w
            continue
    return s  

In [31]:
# 交易回测
money = 10000
b_dt = buy(df)
print(b_dt)
if not b_dt:
    print('无符合条件指标，请更换股票')
else:
    s_date = df['日期'][1]
    for k in range(len(b_dt)):
        b = b_dt[k]
        if df['日期'][b]>s_date:
            #print(str(df['日期'][b]) + ' | ' + str(df['CCI'][b]))
            b_sym = float(((money/df['收盘'][b])//100)*100)
            money = money-(b_sym*df['收盘'][b])
            print('买入： '+ str(df['日期'][b]) + ' | ' + str(df['收盘'][b]) + ' | ' + str(b_sym) + ' | ' + str(money))
            s = sale(b)
            money = money+(b_sym*df['收盘'][s])
            b_sym = 0
            s_date = df['日期'][s]
            print('买出： '+ str(df['日期'][s]) + ' | ' + str(df['收盘'][s]) + ' | '+ str(b_sym) + '     | ' + str(money))
        else:
            continue

[123, 239, 324]
买入： 2024-04-10 | 6.66 | 1500.0 | 10.0
买出： 2024-04-16 | 6.0 | 0     | 9010.0
买入： 2024-09-27 | 6.75 | 1300.0 | 235.0
买出： 2024-11-15 | 7.05 | 0     | 9400.0
买入： 2025-02-11 | 7.47 | 1200.0 | 436.0
买出： 2025-03-25 | 7.69 | 0     | 9664.0


##### 